In [57]:
import numpy as np
import pandas as pd

from pandas.io.json import json_normalize
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [58]:
CLIENT_ID='0QCVQVI22IK3LA2QJLQUXC4Q134AE42HGA0EXUI42ZMAZOVJ'
CLIENT_SECRET='1TH4FCNFM4KLGUBFI0NV1Y5UTJEW5V52I3021ZZOEPVJJ3AV'
VERSION='20180605'
LIMIT=250
cities = ["Melbourne", 'Sydney', 'Adelaide', 'Perth', 'Brisbane']
result={}

In [59]:
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4d4b7105d754a06374d81259") #food category id
    result[city] = requests.get(url).json()
    

In [60]:
df_ven={}
for city in cities:
    venue = json_normalize(result[city]['response']['groups'][0]['items'])
    df_ven[city] = venue[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_ven[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/srv/conda/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


In [61]:
maps = {}
for city in cities:
    city_lat = np.mean([result[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        result[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([result[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        result[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=12)

    # add markers to map
    for lat, lng, label in zip(df_ven[city]['Lat'], df_ven[city]['Lng'], df_ven[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=4,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Food places in {city} =", result[city]['response']['totalResults'])
    print(" Top 50 will be shown","\n")

Total number of Food places in Melbourne = 140
 Top 50 will be shown 

Total number of Food places in Sydney = 185
 Top 50 will be shown 

Total number of Food places in Adelaide = 112
 Top 50 will be shown 

Total number of Food places in Perth = 137
 Top 50 will be shown 

Total number of Food places in Brisbane = 130
 Top 50 will be shown 



In [62]:
maps[cities[0]]
    

In [63]:
maps[cities[1]]

In [64]:
maps[cities[2]]

In [65]:
maps[cities[3]]

In [66]:
maps[cities[4]]

In [67]:
maps = {}
for city in cities:
    city_lat = np.mean([result[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        result[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([result[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        result[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_ven[city]['Lat'].mean(), df_ven[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_ven[city]['Lat'], df_ven[city]['Lng'], df_ven[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='yellow',
            fill=True,
            fill_color='orange',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='brown',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_ven[city][['Lat','Lng']].values)),"\n")

Melbourne
Mean Distance from Mean coordinates
0.023994787227988614 

Sydney
Mean Distance from Mean coordinates
0.05614091615363626 

Adelaide
Mean Distance from Mean coordinates
0.027040684747774595 

Perth
Mean Distance from Mean coordinates
0.05996060348700527 

Brisbane
Mean Distance from Mean coordinates
0.03603455617005355 



In [68]:
maps[cities[0]]

In [69]:
maps[cities[1]]

In [70]:
maps[cities[2]]

In [71]:
maps[cities[3]]

In [72]:
maps[cities[4]]

 Happy time having your declicious food 